In [42]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

In [3]:
class RegressionDataset:

    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self): # returns no. of samples
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = self.data[idx, :] # 2D array
        target = self.targets[idx]

        return {
            "sample": torch.tensor(sample, dtype=torch.float),
            "target": torch.tensor(target, dtype=torch.long)
        }

In [43]:
data, targets = make_regression(n_samples = 1000)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(data, targets)

In [46]:
train_dataset = RegressionDataset(X_train, y_train)
test_dataset = RegressionDataset(X_test, y_test)

In [47]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = 4,
    num_workers = 2,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = 4,
    num_workers = 2,
    shuffle = True
)

In [48]:
def model(X,w,b):
    return torch.matmul(X,w) + b

In [51]:
W = torch.randn(100,1, requires_grad=True) # features X 1
b = torch.randn(1, requires_grad=True)

learning_rate = 1e-3

for epoch in range(25):
    epoch_loss = 0
    for data in train_loader:
        x_train = data['sample']
        y_train = data['target']

        if W.grad is not None or b.grad is not None:
            W.grad_zero_()
            b.gard_zero_()

        output = model(x_train, W, b)
        loss = torch.mean((y_train.view(-1) - output.view(-1)) ** 2)
        epoch_loss = epoch_loss + loss.item()

        loss.backward()
        
        with torch.no_grad(): # temporarily disables requires_grad
            W = W - learning_rate * W.grad
            b = b - learning_rate * b.grad

        # enable grad again for next iteration
        W.requires_grad_(True)
        b.requires_grad_(True)

    print(f'loss at epoch {epoch+1}: {epoch_loss} and actual loss: {epoch_loss/(epoch+1)}')


loss at epoch 1: 4269387.990112305 and actual loss: 4269387.990112305
loss at epoch 2: 1958529.114868164 and actual loss: 979264.557434082
loss at epoch 3: 999019.3253173828 and actual loss: 333006.44177246094
loss at epoch 4: 548857.6106872559 and actual loss: 137214.40267181396
loss at epoch 5: 323101.8698120117 and actual loss: 64620.373962402344
loss at epoch 6: 194679.63706207275 and actual loss: 32446.606177012127
loss at epoch 7: 120781.4140586853 and actual loss: 17254.487722669328
loss at epoch 8: 77407.69997692108 and actual loss: 9675.962497115135
loss at epoch 9: 50509.958936691284 and actual loss: 5612.217659632365
loss at epoch 10: 33417.24080431461 and actual loss: 3341.7240804314615
loss at epoch 11: 22383.72754383087 and actual loss: 2034.8843221664429
loss at epoch 12: 15071.386861801147 and actual loss: 1255.9489051500957
loss at epoch 13: 10240.390727758408 and actual loss: 787.7223636737236
loss at epoch 14: 7005.133543014526 and actual loss: 500.36668164389476
los

In [52]:
W,b

(tensor([[ 4.4965e-02],
         [-4.9854e-02],
         [ 2.9636e-02],
         [-8.2208e-03],
         [ 6.7810e-02],
         [ 6.2622e+01],
         [-1.6991e-01],
         [ 7.2418e+01],
         [ 8.0089e+01],
         [-2.0931e-01],
         [ 9.8876e-03],
         [-1.2243e-01],
         [ 1.0703e-01],
         [ 1.2064e-01],
         [-6.5793e-02],
         [-6.0248e-02],
         [ 7.1999e-03],
         [-1.7551e-01],
         [ 8.4554e-02],
         [ 1.0229e-01],
         [-1.5142e-01],
         [ 8.6202e-02],
         [ 8.2493e-03],
         [-1.4008e-01],
         [ 1.3779e-01],
         [ 8.1896e-04],
         [ 9.1711e-02],
         [-1.0152e-02],
         [ 1.7783e-01],
         [ 8.2209e-02],
         [ 3.7591e-02],
         [-5.3966e-02],
         [-1.3404e-01],
         [ 8.3271e-02],
         [-7.1914e-02],
         [-1.4184e-01],
         [ 4.6714e-02],
         [ 2.7959e-02],
         [ 4.5342e-02],
         [ 8.1739e+01],
         [-2.8486e-02],
         [ 2.195

In [57]:
outputs = []
labels = []

with torch.no_grad():
    for data in test_loader:
        x_test = data['sample']
        y_test = data['target']
        output = model(x_test, W, b)
        labels.append(y_test)
        outputs.append(output)

In [54]:
len(outputs), len(labels)

(63, 63)

In [58]:
from sklearn.metrics import r2_score

outputs = torch.cat(outputs).view(-1)
labels = torch.cat(labels).view(-1)

print(outputs.shape, labels.shape)

r2_score(
    labels, outputs
) # r2 score lies between 0 and 1, more the better

torch.Size([250]) torch.Size([250])


0.9999505360560945